# Supervised Forest Classification

This code uses a unet architecture with two band observations from Sentinel-1 imagery to predict if a pixel is `forest` or `no forest`. 
This part of the code reads the training data from a folder that already has source imagery and labels stored as numpy array files. These files are created with the code `pre-processing.ipynb`.
This is not an optimal solution for a scalable training. Ideally it should read the input Sentinel-1 and labels on the fly for any geographical region and time. 

In [ ]:
from model import *
from data import *
import matplotlib.pyplot as plt
%matplotlib inline
import glob

## Training

### Generate Training Data

In [ ]:
trainData = trainGenerator('data/train/', flag_multi_class = True, num_image = 100)

### Create Model

In [ ]:
forestModel = unet(input_size = (256,256,2))
# model.summary()
model_checkpoint = ModelCheckpoint('unet_forest.hdf5', monitor='loss',verbose=1, save_best_only=True)

### Train Model

In [ ]:
forestModel.fit_generator(trainData, steps_per_epoch=50, epochs=2, callbacks=[model_checkpoint])

## Testing

### Generate Test Data

In [ ]:
# Clean existing data
files = glob.glob('data/test/predict/*.*')
for f in files:
    os.remove(f)


testData = testGenerator('data/test/', flag_multi_class = True, num_image = 17)

### Predict on Test Data

In [ ]:
results = forestModel.predict_generator(testData, steps = 17, verbose=1)
saveResult('data/test/predict/', results, flag_multi_class = False)

## Visualize Test Predictions and Labels

In [ ]:
fig = plt.figure(figsize=(10, 60))
for i in range(17):
    img = np.load('data/test/' + 'image/' + str(i) + '.npy')
    ax1 = fig.add_subplot(20, 3, i*3+1)
    ax1.imshow(img[:, :, 0])
    mask = np.load(os.path.join('data/test/' + 'label/',"%d.npy"%i))
    ax2 = fig.add_subplot(20, 3, i*3+2)
    ax2.imshow(mask)
    predict = np.load(os.path.join('data/test/' + 'predict/',"%d_predict.npy"%i))
    ax3 = fig.add_subplot(20, 3, i*3+3)
    ax3.imshow(predict)
